# this notebook is improved before 
https://www.kaggle.com/code/sinjeongyeol/spaceshiptitanic-binaryclassification-1/notebook
notebook

## plus skill
### Target encoding with smoothing
### Add feature
### optimize hyperparameter

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('/kaggle/input/spaceship-titanic/train.csv')
test = pd.read_csv('/kaggle/input/spaceship-titanic/test.csv')
submission = pd.read_csv('/kaggle/input/spaceship-titanic/sample_submission.csv')

In [ ]:
feats_cat = []
feats_num = []
for col in train.columns:
    if col in ['PassengerId', 'Transported']:
        continue
    if train[col].dtype == 'object':
        feats_cat += [col]
    else:
        feats_num += [col]
        
feats_cat, feats_num

In [ ]:
train_len = len(train)
dataset = pd.concat([train, test], axis=0).reset_index(drop=True)

In [ ]:
dataset.shape, train.shape, test.shape

### PassengerId
### Add group_cnt, group_num

In [ ]:
train['PassengerId']

In [ ]:
dataset['group_id'] = 0
dataset['group_num'] = 0
for i in range(len(dataset)):
    group_id, id_at_group = map(int, dataset.iloc[i]['PassengerId'].split('_'))
    dataset.loc[i, 'group_id'] = group_id
    dataset.loc[i, 'group_num'] = id_at_group

In [ ]:
dataset = pd.merge(dataset, dataset[['group_id', 'PassengerId']].groupby('group_id').count().reset_index().rename(columns={'PassengerId': 'group_cnt'}), how='left', on='group_id')
dataset = dataset.drop('group_id', axis=1)
dataset.head()

In [ ]:
feats_num += ['group_num', 'group_cnt']

In [ ]:
import seaborn as sns

corr = dataset[feats_num].corr()
sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(1, 2, figsize=(10, 6))
dataset[dataset['Transported'].notnull()][['group_cnt', 'Transported']].groupby('group_cnt').mean().plot.bar(ax=ax[0])
sns.countplot(x='group_cnt', data=dataset[dataset['Transported'].notnull()], hue='Transported', ax=ax[1])

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(10, 6))
dataset[dataset['Transported'].notnull()][['group_num', 'Transported']].groupby('group_num').mean().plot.bar(ax=ax[0])
sns.countplot(x='group_num', data=dataset[dataset['Transported'].notnull()], hue='Transported', ax=ax[1])

#### I'll add this features and decide whether to add it or not after seeing feature importance 